# Hausaufgabe - JSON zu XML und SOLR

In [1]:
# JSON Bibliothek laden
import json

# URL für JSON-API
# Search-Term: OCR
url = "https://www.bibsonomy.org/json/search/OCR?items=1000&duplicates=merged"

In [2]:
# Requests Bibliothek laden
import requests

# URL aufrufen und Daten Speichern
response = requests.get(url)
data = response.json()

In [3]:
# JSON menschenlesbar formatieren
data_as_json = json.dumps(data, indent=2)

In [4]:
# Speichere das formatierte JSON um es ggf. für andere Zwecke nachzunutzen
# with-Block um Datei automatisch zu schließen
json_filename = "bibsonomy.json"
with open(json_filename, "w") as file:
    file.write(data_as_json)

In [5]:
# extrahiere nur die tatsächlichen Einträge
items = data['items']

# extrahiere nur diejenigen Einträge, bei denen es sich um eine echte "Publications" handelt
publications = [item for item in items if item['type'] == 'Publication']
#print(len(publications))

# Zeige einen Eintrag, um die interne Struktur des JSON zu sehen
#print(json.dumps(publications[400], indent=2))

## Hausaufgabe Teil 1 - XML

```
<publications>
    <publication>
        <id>1</id>
        <title>...</title>
        <authors>
            <author>...</author>
            <author>...</author>
        </authors>
        <abstract>...</abstract>
        <publisher>...</publisher>
        <year>...</year>
        <type>...</type>
        <!-- weitere Felder -->
    </publication>
    <publication>
        <id>2</id>
        <!-- usw. -->
    </publication>
    <!-- usw. -->
</publications>
```

In [6]:
# Lade XML-Bibliothek => genauer das escaping Modul
from xml.sax.saxutils import escape

xml_filename = "bibsonomy.xml"

# XML-Header
data_as_xml = '<?xml version="1.0" encoding="UTF-8"?>'

# Wurzelelement
data_as_xml += "<publications>"


for i, item in enumerate(publications):
    ## Eintrag öffnen
    data_as_xml += "<publication>"
    
    ### id
    data_as_xml += "<id>" + str(i+1) + "</id>"
    
    ### title
    if "label" in item:
        data_as_xml += "<title>" + escape(item["label"]) + "</title>"
    
    ### author(s)
    if "author" in item:
        data_as_xml += "<authors>"
        for author in item["author"]:
            #### author
            data_as_xml += "<author>" + escape(author) + "</author>"
        data_as_xml += "</authors>"
    
    ### editor(s)
    if "editor" in item:
        data_as_xml += "<editors>"
        for editor in item["editor"]:
            #### author
            data_as_xml += "<editor>" + escape(editor) + "</editor>"
        data_as_xml += "</editors>"
        
    ### abstract
    if "abstract" in item:
        data_as_xml += "<abstract>" + escape(item["abstract"]) + "</abstract>"
        
    ### publisher
    if "publisher" in item:
        data_as_xml += "<publisher>" + escape(item["publisher"]) + "</publisher>"
        
    ### year
    if "year" in item:
        data_as_xml += "<year>" + escape(item["year"]) + "</year>"
    
    ### isbn
    if "isbn" in item:
        data_as_xml += "<isbn>" + escape(item["isbn"]) + "</isbn>"
    
    ### url
    if "url" in item:
        data_as_xml += "<url>" + escape(item["url"]) + "</url>"
        
    ### ee => electronic entity? => Permalink
    if "ee" in item:
        data_as_xml += "<ee>" + escape(item["ee"]) + "</ee>"
        
    ### bibtexKey
    if "bibtexKey" in item:
        data_as_xml += "<bibtexKey>" + escape(item["bibtexKey"]) + "</bibtexKey>"
    
    ### (bibsonomy-)id
    if "id" in item:
        data_as_xml += "<bibsonomy_id>" + escape(item["id"]) + "</bibsonomy_id>"
    
    ## Eintrag schließen
    data_as_xml += "</publication>"

# Wurzelelement schließen
data_as_xml += "</publications>"

# Speichere den XML-String als XML-Datei

import xml.dom.minidom
with open(xml_filename, "w") as file:
    file.write(data_as_xml)

# Lese den XML-Baum ein und wende prettyXML an

dom = xml.dom.minidom.parse(xml_filename)
pretty_xml = dom.toprettyxml(indent="    ")

# Speichere die Änderungen und zeige geänderten Dateiinhalt
with open(xml_filename, "w") as file:
    file.write(pretty_xml)

with open(xml_filename, "r") as file:   
    print(file.read())

<?xml version="1.0" ?>
<publications>
    <publication>
        <id>1</id>
        <title>OCR-Free Table of Contents Detection in Urdu Books.</title>
        <authors>
            <author>Adnan Ul-Hasan</author>
            <author>Syed Saqib Bukhari</author>
            <author>Faisal Shafait</author>
            <author>Thomas M. Breuel</author>
        </authors>
        <editors>
            <editor>Michael Blumenstein</editor>
            <editor>Umapada Pal</editor>
            <editor>Seiichi Uchida</editor>
        </editors>
        <publisher>IEEE Computer Society</publisher>
        <year>2012</year>
        <isbn>978-1-4673-0868-7</isbn>
        <url>http://dblp.uni-trier.de/db/conf/das/das2012.html#Ul-HasanBSB12</url>
        <ee>http://doi.ieeecomputersociety.org/10.1109/DAS.2012.59</ee>
        <bibtexKey>conf/das/Ul-HasanBSB12</bibtexKey>
        <bibsonomy_id>https://www.bibsonomy.org/bibtex/274e7a0fc429afdfb9e78019c499359e5/dblp</bibsonomy_id>
    </publication>
    <

## Hausaufgabe Teil 2 - SOLR

```
[
  {
    "id" : "978-0641723445",
    "cat" : ["book","hardcover"],
    "name" : "The Lightning Thief",
    "author" : "Rick Riordan",
    "series_t" : "Percy Jackson and the Olympians",
    "sequence_i" : 1,
    "genre_s" : "fantasy",
    "inStock" : true,
    "price" : 12.50,
    "pages_i" : 384
  }
,
  {
    "id" : "978-1423103349",
    "cat" : ["book","paperback"],
    "name" : "The Sea of Monsters",
    "author" : "Rick Riordan",
    "series_t" : "Percy Jackson and the Olympians",
    "sequence_i" : 2,
    "genre_s" : "fantasy",
    "inStock" : true,
    "price" : 6.49,
    "pages_i" : 304
  }
]
```

### Hinweis:
Ich habe erst nach Erledigung der Aufgabe festgestellt, dass wir das ursprüngliche JSON hätten bereinigen können/sollen. Auch wenn ich pop() jetzt nicht verwendet habe, ich hätte prinzipiell keine Schwierigkeiten damit es anzuwenden - auch wenn ich hier jetzt nicht den Beweis erbracht habe. Meine Lösung baut ein eigenes JSON zusammen und könnte Daten aus verschiedenen Quellen einspielen...ist zwar ein umständlicherer Weg, führt aber zum selben Ergebnis.

In [7]:
# Dateinamen vorbereiten
solr_filename = "bibsonomy.solr"

# leere Liste anlegen
data_as_solr = []

# item resetten, da escape() auf die elemente angewendet wurde
publications = [item for item in items if item['type'] == 'Publication']

for i, item in enumerate(publications):
    # Generiere Eintrag
    entry = {
        "id": i+1,
    }
    
    # Reichere Eintrag an
    
    ## label => title
    if "label" in item:
        entry.update({"title": item["label"]})
        
    
    ## author(s)
    authors = []
    if "author" in item:
        for j, author in enumerate(item["author"]):
            ### author
            authors.append(escape(author))
    entry.update({"authors": authors})
    
    ## editor(s)
    editors = []
    if "editor" in item:
        for k, editor in enumerate(item["editor"]):
            ### author
            editors.append(escape(editor))
    entry.update({"editors": editors})
    
    # abstract
    if "abstract" in item:
        entry.update({"abstract": item["abstract"]})
    
    # publisher
    if "publisher" in item:
        entry.update({"publisher": item["publisher"]})
    
    # year
    if "year" in item:
        entry.update({"year": item["year"]})
        
    # isbn
    if "isbn" in item:
        entry.update({"isbn": item["isbn"]})
        
    # url
    if "url" in item:
        entry.update({"url": item["url"]})
    
    # ee
    if "ee" in item:
        entry.update({"ee": item["ee"]})
    
    # bibtexKey
    if "bibtexKey" in item:
        entry.update({"bibtexKey": item["bibtexKey"]})
    
    # (bibsonomy-)id
    if "id" in item:
        entry.update({"bibsonomy_id": item["id"]})
    
    data_as_solr.append(entry)

    
# formatiere einmal sauber
data_as_solr = json.dumps(data_as_solr, indent=2)

# Speichere die SOLR-Liste als SOLR-Datei

with open(solr_filename, "w") as file:
    file.write(data_as_solr)

with open(solr_filename, "r") as file:   
    print(file.read())

[
  {
    "id": 1,
    "title": "OCR-Free Table of Contents Detection in Urdu Books.",
    "authors": [
      "Adnan Ul-Hasan",
      "Syed Saqib Bukhari",
      "Faisal Shafait",
      "Thomas M. Breuel"
    ],
    "editors": [
      "Michael Blumenstein",
      "Umapada Pal",
      "Seiichi Uchida"
    ],
    "publisher": "IEEE Computer Society",
    "year": "2012",
    "isbn": "978-1-4673-0868-7",
    "url": "http://dblp.uni-trier.de/db/conf/das/das2012.html#Ul-HasanBSB12",
    "ee": "http://doi.ieeecomputersociety.org/10.1109/DAS.2012.59",
    "bibtexKey": "conf/das/Ul-HasanBSB12",
    "bibsonomy_id": "https://www.bibsonomy.org/bibtex/274e7a0fc429afdfb9e78019c499359e5/dblp"
  },
  {
    "id": 2,
    "title": "Formclas - a System for OCR Free identification of Forms.",
    "authors": [
      "Frank Dubiel",
      "Andreas Dengel"
    ],
    "editors": [
      "Jonathan J. Hull",
      "Suzanne Liebowitz Taylor"
    ],
    "publisher": "WorldScientific",
    "year": "1996",
    "isbn